In [2]:
!pip install catboost
!pip install ngboost

In [22]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import os
import time

PATH = "/content/drive/MyDrive/dacon/gas/data"
PATH_nonleak = "/content/drive/MyDrive/dacon/gas/data/non_leak"

TRAIN_SOURCE = os.path.join(PATH_nonleak, "df_train.csv")
TEST_SOURCE = os.path.join(PATH_nonleak, "df_test.csv")

df_train = pd.read_csv(TRAIN_SOURCE)
df_test = pd.read_csv(TEST_SOURCE)

categorical = ['구분','시간', 'isHoliday', 'year', '24절기삼복', 'month', 'day', 'weekday']
#categorical = ['구분', 'isHoliday', '24절기삼복']
int_col = ['천문박명 시(저녁)', '천문박명 분(저녁)', '천문박명 시(아침)', '천문박명 분(아침)', '시민박명 시(저녁)', '시민박명 분(저녁)', '시민박명 시(아침)', '시민박명 분(아침)', '항해박명 시(저녁)', '항해박명 분(저녁)', '항해박명 시(아침)', '항해박명 분(아침)', '월출 시', '월출 분', '월중 시', '월중 분', '월몰 시', '월몰 분', '일출 시', '일출 분', '일몰 시', '일몰 분']
to_drop = ['연월일', '공급량']

In [8]:
df_train = pd.read_csv('/content/train_ff.csv')
df_test = pd.read_csv('/content/test_ff.csv')
df_train
categorical = ['일출 시', '구분', '공휴일', 'month', 'day', '시간', 'weekday']

In [23]:
from lightgbm import LGBMRegressor
from xgboost import XGBRFRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import TimeSeriesSplit, KFold, GroupKFold
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from catboost import CatBoostRegressor, Pool

def nmae(true, pred):
    score = np.mean((np.abs(true-pred))/true)
    return score

# TUNING

In [ ]:
df_train[categorical] = df_train[categorical].fillna('-9').astype('int64').astype('category')
df_test[categorical] = df_test[categorical].fillna('-9').astype('int64').astype('category')
df_train[int_col] = df_train[int_col].astype('int64')
df_test[int_col] = df_test[int_col].astype('int64')

In [ ]:
cb_pred = []
for gubun in [0,1,2,3,4,5,6]:  
  cb_pred_tmp = []
  mae_list = []
  for lr in [0.005, 0.01, 0.03]:
    for dep in [2,4,5]:
      print(f"step{step}.. ", end='')
      train = df_train[df_train.구분 == gubun]
      test = df_test[df_test.구분 == gubun].drop(columns=['연월일'])
      train['연월일'] = pd.to_datetime(train['연월일'])
      groups = (train['연월일'].dt.year-2013)*12 + train['연월일'].dt.month
      y_labels = train['공급량']
      X_train = train.drop(columns=['공급량','연월일'])
      cb_pred_local = []

      kf = GroupKFold(5)
      cb = CatBoostRegressor(task_type="GPU", iterations = 12000, learning_rate = lr, depth = dep, eval_metric = 'MAE', loss_function = 'MAE')

      for tr_idx, val_idx in kf.split(X_train,y_labels,groups):
        tr_x, val_x = X_train.iloc[tr_idx], X_train.iloc[val_idx]
        tr_y, val_y = y_labels.iloc[tr_idx], y_labels.iloc[val_idx]
        train_data = Pool(data = tr_x, label = tr_y, cat_features = categorical)
        val_data = Pool(data = val_x, label = val_y, cat_features = categorical)
        cb.fit(train_data, eval_set = val_data, early_stopping_rounds = 3000, use_best_model = True, verbose=False, cat_features=categorical)
        best = cb.best_iteration_
        pred = cb.predict(val_x, ntree_end = best)
        score = nmae(val_y, pred)
        mae_list.append(score)
        print(f'FOLD NMAE = {score}')
        sub_pred = cb.predict(test, ntree_end = best)
        cb_pred_local.append(sub_pred)
      
      mean_nmae = np.mean(np.array(mae_list))
      if best_nmae > mean_nmae:
        print(f"Found best param: nmae:{mean_nmae} with param lr:{lr}, depth:{dep}")
        best_nmae = mean_nmae
        best_param = [lr, dep]
        cb_pred_tmp = cb_pred_local
      step+=1
  cb_pred.append(cb_pred_tmp)
  print(f"\nCOMPANY {gubun} result : NMAE:{best_nmae}, PARAM:{best_param}")

step0.. 

CatBoostError: ignored

In [ ]:
#depth 잡고 이터레이션잡고, 0.01, 0.001, 0.1
catb_model = CatBoostRegressor(task_type="GPU",)
catb_params = {"iterations": ['None', 10000, 12500, 15000],
               "learning_rate": ['None', 0.01, 0.03, 0.05, 0.08, 0.1, 0.2],
               "depth": ['None', 2, 4, 6]}

inner_CV = GroupKFold(n_splits = 5)
outer_CV = GroupKFold(n_splits = 5)

for train_index, test_index in outer_CV.split(X_train1, y_labels1, groups=groups1):
    X_tr, X_tt = X_train1.iloc[train_index], X_train1.iloc[test_index]
    y_tr, y_tt = y_labels1.iloc[train_index], y_labels1.iloc[test_index]
    g_tr, g_tt = groups1.iloc[train_index], groups1.iloc[test_index]
    clf = GridSearchCV(estimator=catb_model, param_grid=catb_params, cv=inner_CV)
    clf.fit(X_tr,y_tr,groups=g_tr)

    pred = clf.predict(X_tt)   
    pred_y.extend(pred)
    true_y.extend(y_tt)
    gc.collect()
clf.best_params_

# MODELING

In [24]:
group1 = [0,1,3,4,5]
group2 = [2,6]
df_train['연월일'] = pd.to_datetime(df_train['연월일'])
df_test['연월일'] = pd.to_datetime(df_test['연월일'])

groups1 = ((df_train[df_train['구분'].isin(group1)]['연월일'].dt.year-2013)*12 + df_train[df_train['구분'].isin(group1)]['연월일'].dt.month).reset_index(drop=True)
y_labels1 = df_train[df_train['구분'].isin(group1)]['공급량']
X_train1 = df_train[df_train['구분'].isin(group1)].drop(columns=['연월일','공급량']).reset_index(drop=True)
X_test1 = df_test[df_test['구분'].isin(group1)].drop(columns=['연월일'])
groups2 = ((df_train[df_train['구분'].isin(group2)]['연월일'].dt.year-2013)*12 + df_train[df_train['구분'].isin(group2)]['연월일'].dt.month).reset_index(drop=True)
y_labels2 = df_train[df_train['구분'].isin(group2)]['공급량']
X_train2 = df_train[df_train['구분'].isin(group2)].drop(columns=['연월일','공급량']).reset_index(drop=True)
X_test2 = df_test[df_test['구분'].isin(group2)].drop(columns=['연월일'])

In [25]:
X_train1[categorical] = X_train1[categorical].fillna('999').astype('int64')
X_test1[categorical] = X_test1[categorical].fillna('999').astype('int64')

non_categorical = [col for col in X_train1.columns.tolist() if (col in categorical)==False]
X_train1[non_categorical] = X_train1[non_categorical].apply(lambda x: pd.to_numeric(x))
X_test1[non_categorical] = X_test1[non_categorical].apply(lambda x: pd.to_numeric(x))

int_col = ['천문박명 시(저녁)', '천문박명 분(저녁)', '천문박명 시(아침)', '천문박명 분(아침)', '시민박명 시(저녁)', '시민박명 분(저녁)', '시민박명 시(아침)', '시민박명 분(아침)', '항해박명 시(저녁)', '항해박명 분(저녁)', '항해박명 시(아침)', '항해박명 분(아침)', '월출 시', '월출 분', '월중 시', '월중 분', '월몰 시', '월몰 분', '일출 시', '일출 분', '일몰 시', '일몰 분',]
X_train1[int_col] = X_train1[int_col].astype('int64')
X_test1[int_col] = X_test1[int_col].astype('int64')
X_test1 = X_test1[X_train1.columns.tolist()]

In [26]:
kf = GroupKFold(5)
cb = CatBoostRegressor(task_type="GPU", iterations = 10000, learning_rate = 0.03, depth = 3, eval_metric = 'MAE', loss_function = 'MAE', random_seed = 42)

cb_pred_1 = []
mae_list = []
for tr_idx, val_idx in kf.split(X_train1,y_labels1,groups1 ):
    tr_x, val_x = X_train1.iloc[tr_idx], X_train1.iloc[val_idx]
    tr_y, val_y = y_labels1.iloc[tr_idx], y_labels1.iloc[val_idx]
    cb.fit(tr_x, tr_y, eval_set=(val_x,val_y), cat_features=categorical, early_stopping_rounds = 1000, use_best_model=True, verbose=2000)
    # train_data = Pool(data = tr_x, label = tr_y, cat_features = categorical)
    # val_data = Pool(data = val_x, label = val_y, cat_features = categorical)
    # cb.fit(train_data, eval_set = val_data, early_stopping_rounds = 2000, use_best_model = True, verbose = 1000)
    best = cb.best_iteration_
    pred = cb.predict(val_x, ntree_end = best)
    mae = nmae(val_y, pred)
    mae_list.append(mae)
    print(f'FOLD NMAE = {mae}')
    sub_pred = cb.predict(X_test1, ntree_end = best)
    cb_pred_1.append(sub_pred)
print(f'NMAE = {np.mean(np.array(mae_list))}')

0:	learn: 655.9485412	test: 822.0374760	best: 822.0374760 (0)	total: 16.4ms	remaining: 2m 44s
2000:	learn: 104.0906673	test: 170.9179046	best: 170.9042586 (1955)	total: 30.2s	remaining: 2m
4000:	learn: 88.5471722	test: 169.7634216	best: 169.7579449 (3990)	total: 1m	remaining: 1m 30s
bestTest = 169.2734499
bestIteration = 4536
Shrink model to first 4537 iterations.
FOLD NMAE = 0.12129095460567653
0:	learn: 718.6408580	test: 577.1645639	best: 577.1645639 (0)	total: 15.7ms	remaining: 2m 36s
2000:	learn: 109.0983139	test: 122.3025701	best: 122.3000389 (1997)	total: 29.8s	remaining: 1m 59s
4000:	learn: 92.6910573	test: 120.4639700	best: 120.3947819 (3488)	total: 1m	remaining: 1m 30s
6000:	learn: 83.8293320	test: 119.2615460	best: 119.2360884 (5970)	total: 1m 30s	remaining: 1m
8000:	learn: 78.4803717	test: 119.0359813	best: 118.9962909 (7787)	total: 2m 1s	remaining: 30.2s
bestTest = 118.9962909
bestIteration = 7787
Shrink model to first 7788 iterations.
FOLD NMAE = 0.12674156799148797
0:	lea

In [27]:
X_train2[categorical] = X_train2[categorical].fillna('999').astype('int64')
X_test2[categorical] = X_test2[categorical].fillna('999').astype('int64')

non_categorical = [col for col in X_train2.columns.tolist() if (col in categorical)==False]
X_train2[non_categorical] = X_train2[non_categorical].apply(lambda x: pd.to_numeric(x))
X_test2[non_categorical] = X_test2[non_categorical].apply(lambda x: pd.to_numeric(x))

int_col = ['천문박명 시(저녁)', '천문박명 분(저녁)', '천문박명 시(아침)', '천문박명 분(아침)', '시민박명 시(저녁)', '시민박명 분(저녁)', '시민박명 시(아침)', '시민박명 분(아침)', '항해박명 시(저녁)', '항해박명 분(저녁)', '항해박명 시(아침)', '항해박명 분(아침)', '월출 시', '월출 분', '월중 시', '월중 분', '월몰 시', '월몰 분', '일출 시', '일출 분', '일몰 시', '일몰 분',]
X_train2[int_col] = X_train2[int_col].astype('int64')
X_test2[int_col] = X_test2[int_col].astype('int64')
X_test2 = X_test2[X_train2.columns.tolist()]

In [28]:
kf = GroupKFold(5)
cb = CatBoostRegressor(task_type="GPU", iterations = 10000, learning_rate = 0.03, depth = 3, eval_metric = 'MAE', loss_function = 'MAE')

cb_pred_2 = []
mae_list2 = []
for tr_idx, val_idx in kf.split(X_train2,y_labels2,groups2):
    tr_x, val_x = X_train2.iloc[tr_idx], X_train2.iloc[val_idx]
    tr_y, val_y = y_labels2.iloc[tr_idx], y_labels2.iloc[val_idx]
    # train_data = Pool(data = tr_x, label = tr_y)
    # val_data = Pool(data = val_x, label = val_y)
    cb.fit(tr_x, tr_y, eval_set=(val_x,val_y), cat_features=categorical, early_stopping_rounds = 1000, use_best_model=True, verbose=2000)
    best = cb.best_iteration_
    pred = cb.predict(val_x, ntree_end = best)
    mae = nmae(val_y, pred)
    mae_list2.append(mae)
    print(f'FOLD NMAE = {mae}')
    sub_pred = cb.predict(X_test2, ntree_end = best)
    cb_pred_2.append(sub_pred)
print(f'NMAE = {np.mean(np.array(mae_list2))}')

0:	learn: 87.7318138	test: 103.4672657	best: 103.4672657 (0)	total: 10.3ms	remaining: 1m 43s
2000:	learn: 16.2581745	test: 26.4700636	best: 26.4681892 (1997)	total: 20.3s	remaining: 1m 21s
4000:	learn: 13.9453409	test: 26.2399359	best: 26.2355926 (3756)	total: 40.8s	remaining: 1m 1s
6000:	learn: 12.6500834	test: 26.0922748	best: 26.0915098 (5984)	total: 1m 1s	remaining: 40.9s
8000:	learn: 11.8119826	test: 25.9952148	best: 25.9777870 (7646)	total: 1m 21s	remaining: 20.4s
bestTest = 25.94319346
bestIteration = 8791
Shrink model to first 8792 iterations.
FOLD NMAE = 0.11567851469897548
0:	learn: 91.9165587	test: 86.7848693	best: 86.7848693 (0)	total: 10.7ms	remaining: 1m 47s
2000:	learn: 17.1193150	test: 24.3685068	best: 24.3631659 (1999)	total: 20.4s	remaining: 1m 21s
4000:	learn: 14.6516727	test: 24.0639483	best: 24.0142981 (3812)	total: 41s	remaining: 1m 1s
bestTest = 24.01429812
bestIteration = 3812
Shrink model to first 3813 iterations.
FOLD NMAE = 0.13618238400087881
0:	learn: 92.04

In [64]:
len((cb_pred_2[0] + cb_pred_2[1] + cb_pred_2[2] + cb_pred_2[3] + cb_pred_2[4])/5)

10800

In [29]:
X_test2['공급량'] = (cb_pred_2[0] + cb_pred_2[1] + cb_pred_2[2] + cb_pred_2[3] + cb_pred_2[4])/5
X_test1['공급량'] = (cb_pred_1[0] + cb_pred_1[1] + cb_pred_1[2] + cb_pred_1[3] + cb_pred_1[4])/5
X_test1['연월일'] = df_test[df_test['구분'].isin(group1)]['연월일']
X_test2['연월일'] = df_test[df_test['구분'].isin(group2)]['연월일']

In [30]:
result = pd.concat([X_test1, X_test2])
result = result[['연월일', '시간', '구분', '공급량']]
result

,연월일,시간,구분,공급량
0,2019-01-01,1,0,1931.501943
1,2019-01-01,2,0,1686.315973
2,2019-01-01,3,0,1546.412880
3,2019-01-01,4,0,1595.826302
4,2019-01-01,5,0,1750.919587
...,...,...,...,...
15115,2019-03-31,20,6,319.121392
15116,2019-03-31,21,6,321.845029
15117,2019-03-31,22,6,309.976870
15118,2019-03-31,23,6,296.359184


In [31]:
PATH = "/content/drive/MyDrive/dacon/gas/data"
submission = pd.read_csv(os.path.join(PATH, "sample_submission.csv"))
submission['연월일'] = submission['일자|시간|구분'].str.split(' ').str[0]
submission['시간'] = submission['일자|시간|구분'].str.split(' ').str[1].astype(int)
submission['구분'] = submission['일자|시간|구분'].str.split(' ').str[2]
submission = submission.drop(columns=['공급량'])
submission['연월일'] = pd.to_datetime(submission['연월일'])
submission['시간'] = submission['시간'].astype(int)
submission['구분'] = submission['구분'].map({'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'G': 5, 'H': 6})
submission = pd.merge(submission, result, how='left', on=['연월일', '시간', '구분'])
submission

,일자|시간|구분,연월일,시간,구분,공급량
0,2019-01-01 01 A,2019-01-01,1,0,1931.501943
1,2019-01-01 02 A,2019-01-01,2,0,1686.315973
2,2019-01-01 03 A,2019-01-01,3,0,1546.412880
3,2019-01-01 04 A,2019-01-01,4,0,1595.826302
4,2019-01-01 05 A,2019-01-01,5,0,1750.919587
...,...,...,...,...,...
15115,2019-03-31 20 H,2019-03-31,20,6,319.121392
15116,2019-03-31 21 H,2019-03-31,21,6,321.845029
15117,2019-03-31 22 H,2019-03-31,22,6,309.976870
15118,2019-03-31 23 H,2019-03-31,23,6,296.359184


In [32]:
pred = submission['공급량']
PATH = "/content/drive/MyDrive/dacon/gas/data"
submission = pd.read_csv(os.path.join(PATH, "sample_submission.csv"))
submission['공급량'] = pred
submission.to_csv(os.path.join("/content/drive/MyDrive/dacon/gas/submission", "2staged_cat_me_byme.csv"), index=False)

In [34]:
def target_val(df):
  df['연월일'] = df['일자|시간|구분'].str.split(' ').str[0]
  df['시간'] = df['일자|시간|구분'].str.split(' ').str[1].astype(int)
  df['구분'] = df['일자|시간|구분'].str.split(' ').str[2]
  df['연월일'] = pd.to_datetime(df['연월일'])
  df['시간'] = df['시간'].astype(int)
  df['구분'] = df['구분'].map({'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'G': 5, 'H': 6})
  df = df.sort_values(by=['연월일', '시간', '구분'])['공급량']
  return df

answer = pd.read_csv('/content/drive/MyDrive/dacon/gas/submission/submission_xgboost.csv')
submission = pd.read_csv('/content/drive/MyDrive/dacon/gas/submission/2staged_cat_me_byme.csv')

nmae(target_val(answer), target_val(submission))

0.12213472648356953